In [1]:
# importing the dependencies
import requests
from config import API_KEY
import pandas as pd
import datetime



In [2]:
requests.__version__


'2.25.1'

In [3]:
#importing the vaccin data
vax_data = pd.read_csv("../Project--COVID-19/Resources/EditedCovidDeath.csv", low_memory=False)
vax_type = pd.read_csv("../Project--COVID-19/Resources/2021VAERSVAX.csv", low_memory=False)
vax_symptoms = pd.read_csv("../Project--COVID-19/Resources/2021VAERSSYMPTOMS.csv", low_memory=False)

In [4]:
vax_data.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,916600,1/1/21,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,NaN,...,None,None,NaN,NaN,2,1/1/21,NaN,Y,NaN,Pcn and bee venom
1,916601,1/1/21,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,NaN,...,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,NaN,NaN,2,1/1/21,NaN,Y,NaN,"""Dairy"""
2,916602,1/1/21,WA,23.0,23.0,NaN,F,NaN,"About 15 minutes after receiving the vaccine, ...",NaN,...,None,None,NaN,NaN,2,1/1/21,NaN,NaN,Y,Shellfish
3,916603,1/1/21,WA,58.0,58.0,NaN,F,NaN,"extreme fatigue, dizziness,. could not lift my...",NaN,...,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","got measles from measel shot, mums from mumps ...",NaN,2,1/1/21,NaN,NaN,NaN,"Diclofenac, novacaine, lidocaine, pickles, tom..."
4,916604,1/1/21,TX,47.0,47.0,NaN,F,NaN,"Injection site swelling, redness, warm to the ...",NaN,...,Na,NaN,NaN,NaN,2,1/1/21,NaN,NaN,NaN,Na


In [5]:
len(vax_data)

34121

In [6]:
vax_data.sample(5)

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
32766,1086882,3/10/21,IL,28.0,NaN,NaN,F,NaN,Anaphylactic reaction; A spontaneous report wa...,NaN,...,NaN,Medical History/Concurrent Conditions: No adve...,NaN,"USMODERNATX, INC.MOD20210",2,3/9/21,NaN,NaN,NaN,NaN
6813,924477,1/6/21,AR,25.0,25.0,NaN,F,NaN,within 15 minutes she said a bit dizzy I didn...,NaN,...,NaN,NaN,NaN,NaN,2,1/6/21,NaN,NaN,Y,NaN
22425,1018126,2/9/21,FL,72.0,72.0,NaN,M,NaN,Patient received vaccination on 2/4. Patient's...,Y,...,chronic conditions listed below,"per wife-diabetes, hypertension, hypercholeste...",NaN,NaN,2,2/9/21,NaN,NaN,NaN,na
15456,1011521,1/9/21,NaN,36.0,35.0,NaN,F,NaN,SkinRash 2 days after vaccine onset of painful...,NaN,...,NaN,NaN,NaN,NaN,2,1/8/21,NaN,NaN,NaN,NaN
24943,1046377,2/22/21,NaN,85.0,85.0,NaN,F,NaN,Administration error - Vaccine was under dilut...,NaN,...,NaN,NaN,NaN,NaN,2,2/22/21,NaN,NaN,NaN,NaN


In [7]:
vax_data.tail()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
34116,1113917,3/19/21,FL,88.0,87.0,NaN,F,NaN,throat closure three weeks after second dose; ...,NaN,...,NaN,Medical History/Concurrent Conditions: Aortic ...,NaN,"USMODERNATX, INC.MOD20210",2,3/12/21,NaN,NaN,Y,NaN
34117,1113920,3/19/21,PA,83.0,NaN,NaN,F,NaN,positive COVID-19 test with no symptoms; Asymp...,NaN,...,NaN,Medical History/Concurrent Conditions: Atrial ...,NaN,USPFIZER INC2021231682,2,3/16/21,NaN,Y,NaN,NaN
34118,1113963,3/19/21,MS,59.0,NaN,NaN,F,NaN,HLH (hemophagocyticlymphohistiocytosis); This ...,Y,...,NaN,Medical History/Concurrent Conditions: Type II...,NaN,USPFIZER INC2021257356,2,3/12/21,NaN,NaN,NaN,NaN
34119,1115045,3/19/21,AZ,57.0,NaN,NaN,M,NaN,Death; A spontaneous report was received from ...,Y,...,NaN,Medical History/Concurrent Conditions: No adve...,NaN,"USMODERNATX, INC.MOD20210",2,3/15/21,NaN,NaN,NaN,NaN
34120,1115348,3/19/21,NC,NaN,85.0,NaN,F,NaN,death; A spontaneous report was received from ...,Y,...,NaN,Comments: List of non-encoded Patient Relevant...,NaN,"USMODERNATX, INC.MOD20210",2,3/12/21,NaN,NaN,NaN,NaN


In [8]:
vax_data[['VAERS_ID', 'RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'SEX',
       'RPT_DATE', 'SYMPTOM_TEXT', 'DIED', 'DATEDIED', 'L_THREAT', 'ER_VISIT',
       'HOSPITAL', 'HOSPDAYS', 'X_STAY', 'DISABLE', 'RECOVD', 'VAX_DATE',
       'ONSET_DATE', 'NUMDAYS', 'LAB_DATA', 'V_ADMINBY', 'V_FUNDBY',
       'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'SPLTTYPE',
       'FORM_VERS', 'TODAYS_DATE', 'BIRTH_DEFECT', 'OFC_VISIT', 'ER_ED_VISIT',
       'ALLERGIES']].isnull().sum()


VAERS_ID            0
RECVDATE            0
STATE            5571
AGE_YRS          3188
CAGE_YR          7330
SEX                 0
RPT_DATE        34058
SYMPTOM_TEXT        2
DIED            32164
DATEDIED        32323
L_THREAT        32862
ER_VISIT        34110
HOSPITAL        29734
HOSPDAYS        31264
X_STAY          34069
DISABLE         33251
RECOVD           2857
VAX_DATE         1499
ONSET_DATE       1863
NUMDAYS          2927
LAB_DATA        19041
V_ADMINBY           0
V_FUNDBY        34057
OTHER_MEDS      13882
CUR_ILL         18052
HISTORY         11746
PRIOR_VAX       32687
SPLTTYPE        25898
FORM_VERS           0
TODAYS_DATE       199
BIRTH_DEFECT    34070
OFC_VISIT       28717
ER_ED_VISIT     28592
ALLERGIES       15534
dtype: int64

In [9]:
len(vax_data.columns)

35

In [45]:
unuseful_cols= [ column for column in vax_data.columns if column in ["RECVDATE","CAGE_YR","CAGE_MO","RPT_DATE","SYMPTOM_TEXT","DATEDIED","L_THREAT","ER_VISIT","HOSPDAYS","X_STAY","DISABLE","LAB_DATA","V_FUNDBY","HISTORY ","PRIOR_VAX","SPLTTYPE","BIRTH_DEFECT","OFC_VISIT", "ER_ED_VISIT"]]
unuseful_cols

['HOSPDAYS']

In [46]:
 vax_data = vax_data.drop(unuseful_cols, axis=1)


In [47]:
len(vax_data.columns)

17

In [48]:
vax_data.columns

Index(['VAERS_ID', 'STATE', 'AGE_YRS', 'SEX', 'DIED', 'HOSPITAL', 'RECOVD',
       'VAX_DATE', 'ONSET_DATE', 'NUMDAYS', 'V_ADMINBY', 'OTHER_MEDS',
       'CUR_ILL', 'HISTORY', 'FORM_VERS', 'TODAYS_DATE', 'ALLERGIES'],
      dtype='object')

In [49]:
def change_col_names (old_name,new_name,data = vax_data):
    '''
    change column names and add it to the dataframe
    '''
    if old_name in data.columns:
        data[new_name] = data.pop(old_name)
    change_col_names("AGE_YRS", "AGE")
    change_col_names("CUR_ILL", "CURRENT ILLNESS")
        
    return data

In [50]:
vax_data.dtypes

VAERS_ID                int64
STATE                  object
AGE_YRS               float64
SEX                    object
DIED                   object
HOSPITAL               object
RECOVD                 object
VAX_DATE       datetime64[ns]
ONSET_DATE     datetime64[ns]
NUMDAYS               float64
V_ADMINBY              object
OTHER_MEDS             object
CUR_ILL                object
HISTORY                object
FORM_VERS               int64
TODAYS_DATE    datetime64[ns]
ALLERGIES              object
dtype: object

In [51]:
dates = ['VAX_DATE', 'ONSET_DATE','TODAYS_DATE']

for date in dates:
    vax_data[date] = pd.to_datetime(vax_data[date])

vax_data

,VAERS_ID,STATE,AGE_YRS,SEX,DIED,HOSPITAL,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,V_ADMINBY,OTHER_MEDS,CUR_ILL,HISTORY,FORM_VERS,TODAYS_DATE,ALLERGIES
0,916600,TX,33.0,F,NaN,NaN,Y,2020-12-28,2020-12-30,2.0,PVT,None,None,None,2,2021-01-01,Pcn and bee venom
1,916601,CA,73.0,F,NaN,NaN,Y,2020-12-31,2020-12-31,0.0,SEN,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,2,2021-01-01,"""Dairy"""
2,916602,WA,23.0,F,NaN,NaN,U,2020-12-31,2020-12-31,0.0,SEN,None,None,None,2,2021-01-01,Shellfish
3,916603,WA,58.0,F,NaN,NaN,Y,2020-12-23,2020-12-23,0.0,WRK,none,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...",2,2021-01-01,"Diclofenac, novacaine, lidocaine, pickles, tom..."
4,916604,TX,47.0,F,NaN,NaN,N,2020-12-22,2020-12-29,7.0,PUB,Na,Na,NaN,2,2021-01-01,Na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34116,1113917,FL,88.0,F,NaN,Y,U,2021-01-13,2021-02-25,43.0,UNK,NaN,NaN,Medical History/Concurrent Conditions: Aortic ...,2,2021-03-12,NaN
34117,1113920,PA,83.0,F,NaN,NaN,Y,2021-02-09,NaT,NaN,UNK,ELIQUIS; SYNTHROID; PRAVASTATIN; DILTIAZEM; DO...,NaN,Medical History/Concurrent Conditions: Atrial ...,2,2021-03-16,NaN
34118,1113963,MS,59.0,F,Y,NaN,N,2021-02-05,2021-02-15,10.0,PVT,NaN,NaN,Medical History/Concurrent Conditions: Type II...,2,2021-03-12,NaN
34119,1115045,AZ,57.0,M,Y,NaN,N,2021-03-02,2021-03-02,0.0,UNK,NaN,NaN,Medical History/Concurrent Conditions: No adve...,2,2021-03-15,NaN


In [52]:
vax_data.dtypes

VAERS_ID                int64
STATE                  object
AGE_YRS               float64
SEX                    object
DIED                   object
HOSPITAL               object
RECOVD                 object
VAX_DATE       datetime64[ns]
ONSET_DATE     datetime64[ns]
NUMDAYS               float64
V_ADMINBY              object
OTHER_MEDS             object
CUR_ILL                object
HISTORY                object
FORM_VERS               int64
TODAYS_DATE    datetime64[ns]
ALLERGIES              object
dtype: object

In [53]:
# total deaths in the data
#Certain deaths are prior to the ivention of the vaccine


vax_data["DIED"].count()

1858

In [54]:
df = vax_data.groupby(["STATE"]).count()["DIED"]



In [55]:
# filter the deaths by only keeping the vaccine date from 12/1/21
vax_data = vax_data.loc[(vax_data['VAX_DATE'] >= '2020-12-1')]

In [56]:
vax_data.head(5)

,VAERS_ID,STATE,AGE_YRS,SEX,DIED,HOSPITAL,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,V_ADMINBY,OTHER_MEDS,CUR_ILL,HISTORY,FORM_VERS,TODAYS_DATE,ALLERGIES
0,916600,TX,33.0,F,NaN,NaN,Y,2020-12-28,2020-12-30,2.0,PVT,None,None,None,2,2021-01-01,Pcn and bee venom
1,916601,CA,73.0,F,NaN,NaN,Y,2020-12-31,2020-12-31,0.0,SEN,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,2,2021-01-01,"""Dairy"""
2,916602,WA,23.0,F,NaN,NaN,U,2020-12-31,2020-12-31,0.0,SEN,None,None,None,2,2021-01-01,Shellfish
3,916603,WA,58.0,F,NaN,NaN,Y,2020-12-23,2020-12-23,0.0,WRK,none,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...",2,2021-01-01,"Diclofenac, novacaine, lidocaine, pickles, tom..."
4,916604,TX,47.0,F,NaN,NaN,N,2020-12-22,2020-12-29,7.0,PUB,Na,Na,NaN,2,2021-01-01,Na


In [57]:
# No. of people died post vaccination
vax_data["DIED"].count()

1858

In [58]:
# Check for duplicates
vax_data.duplicated().sum()

0

In [59]:
vax_type.head()

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_SITE,VAX_NAME
0,916600,COVID19,MODERNA,037K20A,1,LA,COVID19 (COVID19 (MODERNA))
1,916601,COVID19,MODERNA,025L20A,1,RA,COVID19 (COVID19 (MODERNA))
2,916602,COVID19,PFIZER\BIONTECH,EL1284,1,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
3,916603,COVID19,MODERNA,unknown,UNK,NaN,COVID19 (COVID19 (MODERNA))
4,916604,COVID19,MODERNA,NaN,1,LA,COVID19 (COVID19 (MODERNA))


In [60]:
len(vax_type)

34142

In [61]:
vax_type.columns


Index(['VAERS_ID', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES',
       'VAX_SITE', 'VAX_NAME'],
      dtype='object')

In [62]:
vax_type[['VAERS_ID', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES',
       'VAX_SITE', 'VAX_NAME']].isnull().sum()

VAERS_ID               0
VAX_TYPE               0
VAX_MANU               0
VAX_LOT            10813
VAX_DOSE_SERIES       51
VAX_SITE            7627
VAX_NAME               0
dtype: int64

In [63]:
# Drop unrequired columns
vax_type = vax_type.drop(['VAX_ROUTE'],  axis = 1)


KeyError: "['VAX_ROUTE'] not found in axis"

In [64]:
vax_type.duplicated().sum()

0

In [65]:
vax_type[vax_type.duplicated() == True]

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_SITE,VAX_NAME


In [66]:
vax_type = vax_type.drop_duplicates()

In [67]:
vax_type.duplicated().sum()

0

In [68]:
vax_type['VAX_NAME'].unique()

array(['COVID19 (COVID19 (MODERNA))',
       'COVID19 (COVID19 (PFIZER-BIONTECH))',
       'COVID19 (COVID19 (UNKNOWN))', 'COVID19 (COVID19 (JANSSEN))'],
      dtype=object)

In [69]:
vax_type['VAX_TYPE'].unique()

array(['COVID19'], dtype=object)

In [70]:
vax_type = vax_type[vax_type['VAX_TYPE'] == 'COVID19']
vax_type

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_SITE,VAX_NAME
0,916600,COVID19,MODERNA,037K20A,1,LA,COVID19 (COVID19 (MODERNA))
1,916601,COVID19,MODERNA,025L20A,1,RA,COVID19 (COVID19 (MODERNA))
2,916602,COVID19,PFIZER\BIONTECH,EL1284,1,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
3,916603,COVID19,MODERNA,unknown,UNK,NaN,COVID19 (COVID19 (MODERNA))
4,916604,COVID19,MODERNA,NaN,1,LA,COVID19 (COVID19 (MODERNA))
...,...,...,...,...,...,...,...
34625,1113917,COVID19,MODERNA,01120A,1,NaN,COVID19 (COVID19 (MODERNA))
34626,1113920,COVID19,PFIZER\BIONTECH,Unknown,1,NaN,COVID19 (COVID19 (PFIZER-BIONTECH))
34627,1113963,COVID19,PFIZER\BIONTECH,EL9264,2,NaN,COVID19 (COVID19 (PFIZER-BIONTECH))
34628,1115045,COVID19,MODERNA,NaN,2,NaN,COVID19 (COVID19 (MODERNA))


In [71]:
vax_type.dtypes

VAERS_ID            int64
VAX_TYPE           object
VAX_MANU           object
VAX_LOT            object
VAX_DOSE_SERIES    object
VAX_SITE           object
VAX_NAME           object
dtype: object

In [72]:
vax_symptoms.head()
len(vax_symptoms)

48110

In [73]:
vax_symptoms.columns

Index(['VAERS_ID', 'SYMPTOM1', 'SYMPTOM2', 'SYMPTOM3'], dtype='object')

In [74]:
vax_symptoms[['VAERS_ID', 'SYMPTOM1', 'SYMPTOM2', 'SYMPTOM3', 'SYMPTOM4', 'SYMPTOM5']].isnull().sum()

KeyError: "['SYMPTOM4', 'SYMPTOM5'] not in index"

In [75]:
vax_symptoms = vax_symptoms.drop(["SYMPTOMVERSION1","SYMPTOMVERSION2","SYMPTOMVERSION3","SYMPTOMVERSION4","SYMPTOMVERSION5", "SYMPTOM4", "SYMPTOM5"], axis =1)

KeyError: "['SYMPTOMVERSION1' 'SYMPTOMVERSION2' 'SYMPTOMVERSION3' 'SYMPTOMVERSION4'\n 'SYMPTOMVERSION5' 'SYMPTOM4' 'SYMPTOM5'] not found in axis"

In [76]:
vax_symptoms.dtypes

VAERS_ID     int64
SYMPTOM1    object
SYMPTOM2    object
SYMPTOM3    object
dtype: object

In [77]:
vax_symptoms.duplicated().sum()

0

In [78]:
final_df = vax_data.merge(vax_type,on='VAERS_ID').merge(vax_symptoms,on='VAERS_ID')

In [79]:
final_df

,VAERS_ID,STATE,AGE_YRS,SEX,DIED,HOSPITAL,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,...,ALLERGIES,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_SITE,VAX_NAME,SYMPTOM1,SYMPTOM2,SYMPTOM3
0,916600,TX,33.0,F,NaN,NaN,Y,2020-12-28,2020-12-30,2.0,...,Pcn and bee venom,COVID19,MODERNA,037K20A,1,LA,COVID19 (COVID19 (MODERNA)),Dysphagia,Epiglottitis,NaN
1,916601,CA,73.0,F,NaN,NaN,Y,2020-12-31,2020-12-31,0.0,...,"""Dairy""",COVID19,MODERNA,025L20A,1,RA,COVID19 (COVID19 (MODERNA)),Anxiety,Dyspnoea,NaN
2,916602,WA,23.0,F,NaN,NaN,U,2020-12-31,2020-12-31,0.0,...,Shellfish,COVID19,PFIZER\BIONTECH,EL1284,1,LA,COVID19 (COVID19 (PFIZER-BIONTECH)),Chest discomfort,Dysphagia,Pain in extremity
3,916603,WA,58.0,F,NaN,NaN,Y,2020-12-23,2020-12-23,0.0,...,"Diclofenac, novacaine, lidocaine, pickles, tom...",COVID19,MODERNA,unknown,UNK,NaN,COVID19 (COVID19 (MODERNA)),Dizziness,Fatigue,Mobility decreased
4,916604,TX,47.0,F,NaN,NaN,N,2020-12-22,2020-12-29,7.0,...,Na,COVID19,MODERNA,NaN,1,LA,COVID19 (COVID19 (MODERNA)),Injection site erythema,Injection site pruritus,Injection site swelling
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46521,1113920,PA,83.0,F,NaN,NaN,Y,2021-02-09,NaT,NaN,...,NaN,COVID19,PFIZER\BIONTECH,Unknown,1,NaN,COVID19 (COVID19 (PFIZER-BIONTECH)),Nasal discomfort,Night sweats,Platelet count
46522,1113920,PA,83.0,F,NaN,NaN,Y,2021-02-09,NaT,NaN,...,NaN,COVID19,PFIZER\BIONTECH,Unknown,1,NaN,COVID19 (COVID19 (PFIZER-BIONTECH)),Rash,SARS-CoV-2 test,NaN
46523,1113963,MS,59.0,F,Y,NaN,N,2021-02-05,2021-02-15,10.0,...,NaN,COVID19,PFIZER\BIONTECH,EL9264,2,NaN,COVID19 (COVID19 (PFIZER-BIONTECH)),Haemophagocytic lymphohistiocytosis,SARS-CoV-2 test,NaN
46524,1115045,AZ,57.0,M,Y,NaN,N,2021-03-02,2021-03-02,0.0,...,NaN,COVID19,MODERNA,NaN,2,NaN,COVID19 (COVID19 (MODERNA)),Death,NaN,NaN


In [81]:
final_df.to_csv("Final_Vaccine_Data.csv")